In [ ]:
import sys
# This makes the gdc19 package avaialble
sys.path.append('../')
import gdc19
# Be sure to set your working directory!!!
# this is the path to the Google Drive folder
gdc19.setup_bane()

In [ ]:
import vtki
import omfvtk
import PVGeo

-----

# Initial Inspection

This notebook loads up all the data and works through an initial inspection.

In [ ]:
proj = omfvtk.load_project(gdc19.get_omf_project_filename())
proj

In [ ]:
topo = proj['land_surface']
topo

# Integrated Scene

Now use `vtki` to create and integrated rendering of all the available data


In [ ]:
p = vtki.BackgroundPlotter()
# p.add_bounds_axes()

In [ ]:
p.add_mesh(proj['temperature'], name='temp', 
           cmap='coolwarm', point_size=10, 
           render_points_as_spheres=True, stitle='Temperature')
p.add_mesh(proj['negro_mag_fault'], name='negro_mag_fault',
           show_edges=False, color='blue', opacity=0.5)
p.add_mesh(proj['opal_mound_fault'], name='opal_mound_fault',
           show_edges=False, color='orange', opacity=0.5)

In [ ]:
# We can update this cell to change the texture
p.add_mesh(topo, 
           texture='geo_aer', 
           show_edges=False, 
           name='topo', 
           reset_camera=False)

In [ ]:
# add the temperature boundaries
p.add_mesh(proj['temp_175c'], name='temp_175c', 
           show_edges=False, color='aquamarine',
           style='wireframe')
p.add_mesh(proj['temp_225c'], name='temp_225c', 
           show_edges=False, color='firebrick',
           style='wireframe')

In [ ]:
# Add the shape file stuff
p.add_mesh(proj['boundary'], name='boundary', 
           color='azure', render_lines_as_tubes=False, line_width=10)

In [ ]:
# Add the granitoid boundary
p.add_mesh(proj['top_granitoid'], name='top_granitoid',
           show_edges=False, style='wireframe')

In [ ]:
# Add the granitoid boundary
#--
# Lets run a PVGeo filter to extract the topo surface from
#    the granitoid surface
ext = PVGeo.grids.ExtractTopography(
        remove=True, # remove the inactive cells
        tolerance=10.0 # buffer around the topo surface
        ).Apply(proj['top_granitoid'], topo)
granitoid = ext
#--
# Now plot it!
p.add_mesh(granitoid, name='top_granitoid',
           show_edges=False, style='surface',
           color='grey', opacity=0.75, reset_camera=False,
          )

In [ ]:
# _ = p.screenshot('../integrated_view.png')

In [ ]:
import pandas as pd
well_locs = pd.read_csv(gdc19.get_well_path('well_location_from_earth_model.csv'))
well_locs.head()

In [ ]:
foo = PVGeo.pointsToPolyData(well_locs[['x', 'y', 'z (land surface)']].values)
p.add_mesh(foo, name='well_locations', point_size=10, color='orange')
